In [26]:
import sys
from math import *
import math
sys.set_int_max_str_digits(999999999)
def fct(n=0):
    yield 2**n
    yield from fct(n+1)
gen = fct(0)
print(f"{sys.getsizeof(gen) =}")
for i,j in zip(range(33),gen):
    print(f"2^{i} = ",j,end=' | '+"\n"*(i%4==0))

sys.getsizeof(gen) =108
2^0 =  1 | 
2^1 =  2 | 2^2 =  4 | 2^3 =  8 | 2^4 =  16 | 
2^5 =  32 | 2^6 =  64 | 2^7 =  128 | 2^8 =  256 | 
2^9 =  512 | 2^10 =  1024 | 2^11 =  2048 | 2^12 =  4096 | 
2^13 =  8192 | 2^14 =  16384 | 2^15 =  32768 | 2^16 =  65536 | 
2^17 =  131072 | 2^18 =  262144 | 2^19 =  524288 | 2^20 =  1048576 | 
2^21 =  2097152 | 2^22 =  4194304 | 2^23 =  8388608 | 2^24 =  16777216 | 
2^25 =  33554432 | 2^26 =  67108864 | 2^27 =  134217728 | 2^28 =  268435456 | 
2^29 =  536870912 | 2^30 =  1073741824 | 2^31 =  2147483648 | 2^32 =  4294967296 | 


In [35]:
class IPv4:
    def __init__(self, part1, part2, part3, part4, cidr=None, mask=None):
        self.part1 = part1
        self.part2 = part2
        self.part3 = part3
        self.part4 = part4
        if cidr != None:
            self.cidr = cidr
            self.mask = self.cidrToMask(cidr)
        elif mask != None:
            self.mask = mask
            self.cidr = self.maskToCIDR()
        else:
            self.cidr = 0
            self.mask = "255.255.255.255"
        self.rangeSize = 2**(32-self.cidr)
        self.usableRangeSize = self.rangeSize - 2
        range_addresses = self.subnetRangeAdresses()
        self.first_sb_adress = range_addresses[0]
        self.last_sb_adress = range_addresses[1]

    def __str__(self):
        return f"{self.part1}.{self.part2}.{self.part3}.{self.part4}"

    def __format__(self, format_spec):
        match format_spec:
            case "cidr":
                return f"{self.part1}.{self.part2}.{self.part3}.{self.part4}/{self.cidr}"
            case "mask":
                return f"{self.part1}.{self.part2}.{self.part3}.{self.part4}:{self.mask}"
        return self.__str__()
    
    def cidrToMask(self,cidr):
        mask = 0
        for i in range(cidr):
            mask |= (1<<(31-i))
        return(f"{mask>>24}.{mask>>16&0xFF}.{mask>>8&0xFF}.{mask&0xFF}")

    def subnetRangeAdresses(self):
        first_sb_adress = (self.part1, self.part2, self.part3, self.part4)
        last_sb_adress = (self.part1, self.part2, self.part3, self.part4)
        mask = self.mask
        mask0, mask8, mask16, mask24 = map(int, mask.split("."))
        size = 0
        offset = 0
        if 32 > self.cidr >= 24:
            size = 2**(32 - self.cidr)
            offset = self.part4 % size
            first_sb_adress = (self.part1, self.part2, self.part3, self.part4 - offset)
            last_sb_adress = (self.part1, self.part2, self.part3, self.part4 - offset + size - 1)
        elif 24 > self.cidr >= 16:
            size = 2**(24 - self.cidr)
            offset = self.part3 % size
            first_sb_adress = (self.part1, self.part2, self.part3 - offset, 0)
            last_sb_adress = (self.part1, self.part2, self.part3 - offset + size - 1, 255)
        elif 16 > self.cidr >= 8:
            size = 2**(16 - self.cidr)
            offset = self.part2 % size
            first_sb_adress = (self.part1, self.part2 - offset, 0, 0)
            last_sb_adress = (self.part1, self.part2 - offset + size - 1, 255, 255)
        elif 8 > self.cidr >= 0:
            size = 2**(8 - self.cidr)
            offset = self.part1 % size
            first_sb_adress = (self.part1 - offset, 0, 0, 0)
            last_sb_adress = (self.part1 - offset + size - 1, 255, 255, 255)
        return (first_sb_adress, last_sb_adress)

    def networkAddress(self):
        mask_parts = list(map(int, self.mask.split('.')))
        network_part1 = self.part1 & mask_parts[0]
        network_part2 = self.part2 & mask_parts[1]
        network_part3 = self.part3 & mask_parts[2]
        network_part4 = self.part4 & mask_parts[3]
        return (network_part1, network_part2, network_part3, network_part4)

    def maskToCIDR(self):
        mask_parts = self.mask.split('.')
        cidr_count = 0
        for part in mask_parts:
            binary_representation = format(int(part), '08b')
            cidr_count += binary_representation.count('1')
        return cidr_count


In [63]:
ip1 = IPv4(165,25,129,0,mask="255.255.192.0")
print(f"{ip1:'cidr'}")
print(ip1.cidr)
print(ip1.mask)
print(ip1.rangeSize)
print(ip1.usableRangeSize)
print(ip1.cidrToMask(18))

165.25.129.0
18
255.255.192.0
16384
16382
255.255.192.0


In [64]:
ip23 = IPv4(100,19,38,52,23)
print(ip23.first_sb_adress,ip23.last_sb_adress,sep='->')

(100, 19, 38, 0)->(100, 19, 39, 255)


In [65]:
ip23 = IPv4(100,19,38,52,18)
print(ip23.networkAddress())

(100, 19, 0, 0)
